Name : Gandluru Mohammed Yaseen  
Reg.No : 12409548  
M.Tech specialized in AI and ML  
Lovely Professional University  
Mobile: 8328377285  
mail : gandlurumohammedyaseen@gmail.com  
LinkedIn: [yaseeng-md](https://www.linkedin.com/in/yaseeng-md/)  
GitHub: [yaseeng-md](http://www.github.com/yaseeng-md)


# Import all required Libraries 

In [3]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
import numpy as np
import torch
import cv2
from PIL import Image,Image 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from transformers import TrOCRProcessor,VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments,default_data_collator
print("Is GPU Avilable ?",torch.cuda.is_available())
print("GPU :", torch.cuda.get_device_name(torch.cuda.current_device()))

2025-05-15 08:29:27.634422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747297767.816356      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747297767.870693      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Is GPU Avilable ? True
GPU : Tesla P100-PCIE-16GB


# Download the Dataset

In [4]:
from datasets import load_dataset
ds = load_dataset("Teklia/IAM-line")

README.md:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/73.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6482 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2915 [00:00<?, ? examples/s]

In [5]:
train_dataset = ds["train"]
eval_dataset = ds["validation"]

#### Process the dataset according to TrOCR model

In [6]:
from transformers import TrOCRProcessor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

# Preprocess the Data

In [7]:
def preprocess(example):
    image = example["image"].convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values[0]
    labels = processor.tokenizer(
        example["text"],
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors="pt"
    ).input_ids[0]

    return {"pixel_values": pixel_values, "labels": labels}

In [8]:
train_dataset = train_dataset.map(preprocess, remove_columns=["image", "text"])
eval_dataset = eval_dataset.map(preprocess, remove_columns=["image", "text"])

Map:   0%|          | 0/6482 [00:00<?, ? examples/s]

Map:   0%|          | 0/976 [00:00<?, ? examples/s]

In [9]:
train_dataset.set_format(type="torch")
eval_dataset.set_format(type="torch")

In [10]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 6482
Number of validation examples: 976


# Model Training

#### Import model from HuggingFace

In [11]:
from transformers import VisionEncoderDecoderModel
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Hyper Paramters and Config the Model

In [12]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4
model.config.epochs = 3

In [13]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    eval_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=1000,
    eval_steps=200,
    num_train_epochs = 10
)

In [17]:
pip install evaluate jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.6 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


#### Define Metrics

In [19]:
import evaluate
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

In [21]:
def compute_metrics(eval_preds):
    pred_ids, label_ids = eval_preds
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer, "wer": wer}

#### Use wandb for Model Evaluation and monitoring models performnace 

In [22]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb_api_key")
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr-checkpoints",          
    predict_with_generate=True,                
    eval_strategy="steps",               
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=True,                                 
    logging_steps=2,
    save_steps=1000,
    eval_steps=200,
    num_train_epochs=10,
    save_total_limit=3,                        
    report_to="wandb",                         
    run_name="trocr-iam-line-experiment"       
)

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=processor.tokenizer,          
    compute_metrics=compute_metrics,          
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator       
)

/tmp/ipykernel_35/687055305.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


#### Model Training

In [25]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss,Cer,Wer
200,0.581900,0.388450,0.626705,0.731910
400,0.294900,0.349067,0.643462,0.757865
600,0.224600,0.305517,0.573347,0.711236
800,0.214500,0.327735,0.583258,0.670000
1000,0.206700,0.298500,0.771189,0.847865
1200,0.166900,0.290580,0.574060,0.693933
1400,0.181700,0.302091,0.594120,0.705730
1600,0.159500,0.268229,0.613966,0.743146
1800,0.162100,0.272253,0.564363,0.644157
2000,0.125300,0.267065,0.565147,0.654494


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


KeyboardInterrupt: 